## Problem sidi chenhou

In [46]:
# !pip install emoji

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import emoji
from binance.client import Client
from datetime import datetime
from sklearn.preprocessing import StandardScaler
import csv

In [47]:
input_file = 'G-D.csv'
output_file = 'G-D-clean.csv'

with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile, quoting=csv.QUOTE_MINIMAL)

    header = next(reader) 
    writer.writerow(header)

    buffer = []
    for line in infile:
        line = line.strip('\n')

        buffer.append(line)
        joined_line = ' '.join(buffer)

        try:
            row = next(csv.reader([joined_line]))

            if len(row) == len(header):
                writer.writerow(row)
                buffer = []  
            else:
                continue
        except Exception:
            continue 

    if buffer:
        print("Warning: leftover data that didn't parse cleanly:", buffer)

In [48]:
import pandas as pd

data = pd.read_csv('G-D-clean.csv')

data.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ChefSam,Sunshine State,Culinarian | Hot Sauce Artisan | Kombucha Brew...,2011-03-23 03:50:13,4680,2643,6232,False,2023-03-01 23:59:59,Which #bitcoin books should I think about read...,['bitcoin'],Twitter for iPhone,False
1,Roy⚡️,NaN,Truth-seeking pleb 📚 • Science 🧪 • Nature 🌱☀️ ...,2022-01-30 17:41:41,770,1145,9166,False,2023-03-01 23:59:47,"@ThankGodForBTC I appreciate the message, but ...",['Bitcoin'],Twitter for iPhone,False
2,Ethereum Yoda,NaN,UP or DOWN... . . . . Price matters NOT.,2022-07-24 04:50:18,576,1,0,False,2023-03-01 23:59:42,#Ethereum price update: #ETH $1664.02 USD #B...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False
3,Viction,"Paris, France",https://t.co/8M3rgdjwEe #bitcoin #blockchain ...,2010-03-26 10:15:26,236,1829,2195,False,2023-03-01 23:59:36,CoinDashboard v3.0 is here Available on ios an...,['Bitcoin'],Twitter for Android,False
4,Rosie,London,"The flower language of jasmine is loyalty, res...",2013-02-16 09:57:56,12731,46,134,False,2023-03-01 23:59:32,#Bitcoin Short Term Fractal (4H)💥 In lower ti...,"['Bitcoin', 'BTC']",Twitter Web App,False


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169820 entries, 0 to 169819
Data columns (total 13 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_name         169818 non-null  object
 1   user_location     84162 non-null   object
 2   user_description  158756 non-null  object
 3   user_created      169820 non-null  object
 4   user_followers    169820 non-null  int64 
 5   user_friends      169820 non-null  int64 
 6   user_favourites   169820 non-null  int64 
 7   user_verified     169820 non-null  bool  
 8   date              169820 non-null  object
 9   text              169820 non-null  object
 10  hashtags          169820 non-null  object
 11  source            169820 non-null  object
 12  is_retweet        169820 non-null  bool  
dtypes: bool(2), int64(3), object(8)
memory usage: 14.6+ MB


In [50]:
data.isnull().sum()

user_name               2
user_location       85658
user_description    11064
user_created            0
user_followers          0
user_friends            0
user_favourites         0
user_verified           0
date                    0
text                    0
hashtags                0
source                  0
is_retweet              0
dtype: int64

In [51]:
data.describe()

,user_followers,user_friends,user_favourites
count,1.698200e+05,169820.000000,1.698200e+05
mean,1.059803e+04,771.285862,6.244843e+03
std,1.308698e+05,2677.149477,2.183433e+04
min,0.000000e+00,0.000000,0.000000e+00
25%,1.190000e+02,9.000000,1.100000e+01
50%,5.460000e+02,122.000000,2.810000e+02
75%,1.956000e+03,606.000000,3.303000e+03
max,1.878937e+07,254276.000000,1.083014e+06


In [52]:
data.describe(include='object')

,user_name,user_location,user_description,user_created,date,text,hashtags,source
count,169818,84162,158756,169820,169820,169820,169820,169820
unique,36290,9789,33302,37127,142500,167798,41604,722
top,Ethereum Yoda,USA,UP or DOWN... . . . . Price matters NOT.,2022-07-24 04:50:18,2023-03-01 14:00:03,Follow @TiTiProtocol👇 The First-ever Use-To-Ea...,['Bitcoin'],Twitter Web App
freq,8721,2146,8714,8714,14,48,29314,53712


In [53]:
data['user_created'] = pd.to_datetime(data['user_created'], errors='coerce')
data['date'] = pd.to_datetime(data['date'], errors='coerce')

data = data.drop(columns=['user_location'])

In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169820 entries, 0 to 169819
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   user_name         169818 non-null  object        
 1   user_description  158756 non-null  object        
 2   user_created      169820 non-null  datetime64[ns]
 3   user_followers    169820 non-null  int64         
 4   user_friends      169820 non-null  int64         
 5   user_favourites   169820 non-null  int64         
 6   user_verified     169820 non-null  bool          
 7   date              169820 non-null  datetime64[ns]
 8   text              169820 non-null  object        
 9   hashtags          169820 non-null  object        
 10  source            169820 non-null  object        
 11  is_retweet        169820 non-null  bool          
dtypes: bool(2), datetime64[ns](2), int64(3), object(5)
memory usage: 13.3+ MB


In [55]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[^\w\s\-]", "", text)  
    text = re.sub(r"\s+", " ", text).strip()
    return text

text_columns = ['user_name', 'text', 'user_description', 'source']

for col in text_columns:
    data[col] = data[col].apply(lambda x: clean_text(x) if pd.notna(x) else None)
    data[col] = data[col].apply(lambda x: None if pd.isna(x) or str(x).strip() == '' else x)
    most_common = data[col].mode()[0]
    data[col] = data[col].fillna(most_common)

In [56]:
print(data.isnull().sum())
print(data.shape)

user_name           0
user_description    0
user_created        0
user_followers      0
user_friends        0
user_favourites     0
user_verified       0
date                0
text                0
hashtags            0
source              0
is_retweet          0
dtype: int64
(169820, 12)


In [57]:
data.head()

,user_name,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,chefsam,culinarian hot sauce artisan kombucha brewer n...,2011-03-23 03:50:13,4680,2643,6232,False,2023-03-01 23:59:59,which books should i think about reading next,['bitcoin'],twitter for iphone,False
1,roy,truth-seeking pleb science nature stacker free...,2022-01-30 17:41:41,770,1145,9166,False,2023-03-01 23:59:47,i appreciate the message but not a fan of the ...,['Bitcoin'],twitter for iphone,False
2,ethereum yoda,up or down price matters not,2022-07-24 04:50:18,576,1,0,False,2023-03-01 23:59:42,price update 166402 usd 0070428 follow for rec...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",twitter web app,False
3,viction,up or down price matters not,2010-03-26 10:15:26,236,1829,2195,False,2023-03-01 23:59:36,coindashboard v30 is here available on ios and...,['Bitcoin'],twitter for android,False
4,rosie,the flower language of jasmine is loyalty resp...,2013-02-16 09:57:56,12731,46,134,False,2023-03-01 23:59:32,short term fractal 4h in lower timeframe price...,"['Bitcoin', 'BTC']",twitter web app,False


In [58]:
data.describe()

,user_created,user_followers,user_friends,user_favourites,date
count,169820,1.698200e+05,169820.000000,1.698200e+05,169820
mean,2019-07-29 12:13:19.107308032,1.059803e+04,771.285862,6.244843e+03,2023-03-01 19:18:17.811100160
min,2006-09-04 17:48:16,0.000000e+00,0.000000,0.000000e+00,2023-02-25 20:49:09
25%,2017-12-07 10:53:57,1.190000e+02,9.000000,1.100000e+01,2023-02-28 04:22:43
50%,2021-05-25 22:45:21,5.460000e+02,122.000000,2.810000e+02,2023-03-01 16:45:07.500000
75%,2022-07-11 05:24:30,1.956000e+03,606.000000,3.303000e+03,2023-03-03 12:21:07.750000128
max,2023-03-05 23:20:37,1.878937e+07,254276.000000,1.083014e+06,2023-03-05 23:59:56
std,NaN,1.308698e+05,2677.149477,2.183433e+04,NaN


In [59]:
data.describe(include='object')

,user_name,user_description,text,hashtags,source
count,169820,169820,169820,169820,169820
unique,35105,31481,141531,41604,719
top,ethereum yoda,up or down price matters not,start trading with a world leading prop firm,['Bitcoin'],twitter web app
freq,9791,28936,355,29314,53727


In [60]:
X_num = data[['user_followers', 'user_friends', 'user_favourites']]

scaler = StandardScaler()
X_num_scaled = scaler.fit_transform(X_num)

In [61]:
data.describe()

,user_created,user_followers,user_friends,user_favourites,date
count,169820,1.698200e+05,169820.000000,1.698200e+05,169820
mean,2019-07-29 12:13:19.107308032,1.059803e+04,771.285862,6.244843e+03,2023-03-01 19:18:17.811100160
min,2006-09-04 17:48:16,0.000000e+00,0.000000,0.000000e+00,2023-02-25 20:49:09
25%,2017-12-07 10:53:57,1.190000e+02,9.000000,1.100000e+01,2023-02-28 04:22:43
50%,2021-05-25 22:45:21,5.460000e+02,122.000000,2.810000e+02,2023-03-01 16:45:07.500000
75%,2022-07-11 05:24:30,1.956000e+03,606.000000,3.303000e+03,2023-03-03 12:21:07.750000128
max,2023-03-05 23:20:37,1.878937e+07,254276.000000,1.083014e+06,2023-03-05 23:59:56
std,NaN,1.308698e+05,2677.149477,2.183433e+04,NaN


In [62]:
data.describe(include='object')

,user_name,user_description,text,hashtags,source
count,169820,169820,169820,169820,169820
unique,35105,31481,141531,41604,719
top,ethereum yoda,up or down price matters not,start trading with a world leading prop firm,['Bitcoin'],twitter web app
freq,9791,28936,355,29314,53727


In [63]:
api_key = "TLXuzjmtV05vOkRYBaeQXWg3p3A6FbsEEVur2ixrJB7bBjY76G0IS1sp4ALhKQjq"
api_secret = "7DNgiq61ajYq7Ul6l8wW5sfUqgSr742PHHqMZgh3Gcn0hnrssvMqKWF80XhKsUtT"

client = Client(api_key, api_secret)

symbol = "BTCUSDT"
interval = Client.KLINE_INTERVAL_1MINUTE
start_time = "2023-02-25 00:00:00"
end_time = "2023-03-05 23:59:59"

klines = client.get_historical_klines(symbol, interval, start_str=start_time, end_str=end_time)

df_price = pd.DataFrame(klines, columns=[
    "open_time", "open", "high", "low", "close", "volume",
    "close_time", "quote_asset_volume", "num_trades",
    "taker_buy_base_volume", "taker_buy_quote_volume", "ignore"
])

df_price["date"] = pd.to_datetime(df_price["open_time"], unit="ms")
df_price["price"] = df_price["close"].astype(float)
df_price = df_price[["date", "price"]]


C:\Users\hp\AppData\Local\Temp\ipykernel_21400\3827015628.py:11: DeprecationWarning: Parsing dates involving a day of month without a year specified is ambiguious
and fails to parse leap day. The default behavior will change in Python 3.15
to either always raise an exception or to use a different default year (TBD).
To avoid trouble, add a specific year to the input & format.
See https://github.com/python/cpython/issues/70647.
  klines = client.get_historical_klines(symbol, interval, start_str=start_time, end_str=end_time)


In [64]:
tweets = data.copy()
tweets['date_minute'] = tweets['date'].dt.floor('min')
tweets_prices = pd.merge(tweets, df_price, how='left', left_on='date_minute', right_on='date')
tweets_prices.drop(columns=['date_minute', 'date_y'], inplace=True)
tweets_prices.rename(columns={'date_x': 'date'}, inplace=True)
tweets_prices[['date', 'text', 'price']].head()

,date,text,price
0,2023-03-01 23:59:59,which books should i think about reading next,23628.97
1,2023-03-01 23:59:47,i appreciate the message but not a fan of the ...,23628.97
2,2023-03-01 23:59:42,price update 166402 usd 0070428 follow for rec...,23628.97
3,2023-03-01 23:59:36,coindashboard v30 is here available on ios and...,23628.97
4,2023-03-01 23:59:32,short term fractal 4h in lower timeframe price...,23628.97


In [65]:
tweets_prices.isnull().sum()

user_name           0
user_description    0
user_created        0
user_followers      0
user_friends        0
user_favourites     0
user_verified       0
date                0
text                0
hashtags            0
source              0
is_retweet          0
price               0
dtype: int64

In [66]:
tweets_prices.head()

,user_name,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,price
0,chefsam,culinarian hot sauce artisan kombucha brewer n...,2011-03-23 03:50:13,4680,2643,6232,False,2023-03-01 23:59:59,which books should i think about reading next,['bitcoin'],twitter for iphone,False,23628.97
1,roy,truth-seeking pleb science nature stacker free...,2022-01-30 17:41:41,770,1145,9166,False,2023-03-01 23:59:47,i appreciate the message but not a fan of the ...,['Bitcoin'],twitter for iphone,False,23628.97
2,ethereum yoda,up or down price matters not,2022-07-24 04:50:18,576,1,0,False,2023-03-01 23:59:42,price update 166402 usd 0070428 follow for rec...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",twitter web app,False,23628.97
3,viction,up or down price matters not,2010-03-26 10:15:26,236,1829,2195,False,2023-03-01 23:59:36,coindashboard v30 is here available on ios and...,['Bitcoin'],twitter for android,False,23628.97
4,rosie,the flower language of jasmine is loyalty resp...,2013-02-16 09:57:56,12731,46,134,False,2023-03-01 23:59:32,short term fractal 4h in lower timeframe price...,"['Bitcoin', 'BTC']",twitter web app,False,23628.97


In [67]:
tweets_prices.to_csv("tweets_with_btc_price.csv", index=False)